# Karpathy Makemore using MLP

[Video](https://www.youtube.com/watch?v=TCH_1BHY58I&t=10s)

Paper [A neural probabilistic language model](https://jmlr.org/papers/volume3/bengio03a/bengio03a.pdf). Bengio et al, 2003.

The paper uses three previous words to predict a fourth word. It uses a vocabulary of 17k words, implemented in a 30-dimensional space.


In [1]:
using Plots
using Flux
using Flux: onehotbatch
using Statistics


In [2]:
# Read names.txt into words array:
f = open("names.txt","r")
    s = read(f,String)
close(f)
words = split(s,"\n")
words[1:10]

10-element Vector{SubString{String}}:
 "emma"
 "olivia"
 "ava"
 "isabella"
 "sophia"
 "charlotte"
 "mia"
 "amelia"
 "harper"
 "evelyn"

In [9]:
# Create character embeddings. We're going to do this a little
# differently, so that we can have the same embeddings AK uses.
# I.e. the index of "." is 0
chars = "abcdefghijklmnopqrstuvwxyz"
stoi = Dict( s => i for (i,s) in enumerate(chars))
stoi['.'] = 0
itos = Dict( i => s for (i,s) in enumerate(chars))
itos[0] = '.'


'.': ASCII/Unicode U+002E (category Po: Punctuation, other)

In [10]:
# Compile dataset for neural net:
block_size = 3 # context length: how many chars to we use to predict next one?
X,Y = [],[]

for w in words[1:5]
    println(w)
    context = zeros(Int64,block_size)
    for ch in string(w,".")
        ix = stoi[ch]
        push!(X,context)
        push!(Y,ix)
        println(join(itos[i] for i in context)," ---> ", itos[ix])
        context = vcat(context[2:end],[ix])
    end
end


emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [13]:
# Build embedding lookup table C.
C = randn(27,2)

27×2 Matrix{Float64}:
 -1.8697    -0.286131
  1.32779   -0.640141
  0.270252  -0.0124062
 -0.552227   0.479913
 -1.23392   -0.460708
 -1.54253   -0.602456
 -0.564617   0.67428
  1.38063    0.104041
  0.61998    1.28132
  0.299722   0.376024
  ⋮         
  1.30014   -0.759306
  1.10377    1.30924
  0.39772    0.3565
  1.18114    0.296153
  0.493363  -1.17184
  0.941814  -1.33601
  2.04603    0.216104
  0.739047  -0.721043
 -0.146573   0.36737